In [3]:
# Import libraries
import pandas as pd
import numpy  as np
import os
from scipy import stats

In [4]:
# Read csv

meta = pd.read_csv("align_overview_sum - align_overview_sum.csv")

In [5]:
# Filter data 

# Conditions: 

# contains speech = yes
# female speak or male speek != no, no
# Arousal_binary != -1
# Valence_binary != -1
# check date != 0 (out-of-hours recording)

# Approx 432 samples after filtering

# data = meta.loc[(meta['check date'] != 0) & (meta['Arousal_binary'] != -1) & (meta['Valence_binary'] != -1)].reset_index(drop=True)
# data['ID'] = data['audio'].str[:5]
# fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
# mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
# data = data.loc[(data['ID'].isin(fspeak)) & (data["female speak"] == "yes") | (data['ID'].isin(mspeak)) & (data["male speak"] == "yes")].reset_index(drop=True)

data = meta.loc[(meta['Arousal_binary'] != -1) & (meta['Valence_binary'] != -1) & (meta['check date'] != 0)].reset_index(drop=True)
data['ID'] = data['audio'].str[:5]
fspeak = ["P_001","P_002","P_003","P_004","P_005","P_006","P_007","P_008","P_009","P_010","P_011","Z_012","P_013"]
mspeak = ["Z_001","Z_002","Z_003","Z_004","Z_005","Z_006","Z_007","Z_008","Z_009","Z_010","Z_011","P_012","Z_013"]
data = data.loc[(data["female speak"] == "yes") & (data["male speak"] == "yes")].reset_index(drop=True)


In [6]:
data

,audio,contain speech,female speak,male speak,female transcript,male transcript,annotate,useful annotates,female chunk number,aligned female chunk number,...,Valence,Arousal_binary,Valence_binary,check annotation,check female transcript,check male transcript,check date,romantic interaction,romantic interaction & both speak,ID
0,P_001\Day_2\Hour_16\07-16-2019 16_44_00,yes,yes,yes,1,1,1,5,2,2,...,100,1,1,1,1,1,1,1,1,P_001
1,P_001\Day_2\Hour_18\07-16-2019 18_24_27,yes,yes,yes,1,1,1,29,18,12,...,94,1,1,1,1,1,1,1,1,P_001
2,P_001\Day_2\Hour_9\07-16-2019 09_00_26,yes,yes,yes,1,1,0,-1,7,-1,...,77,1,1,0,1,1,1,1,1,P_001
3,P_001\Day_3\Hour_16\07-17-2019 16_06_50,yes,yes,yes,1,1,1,13,20,9,...,100,1,1,1,1,1,1,1,1,P_001
4,P_001\Day_3\Hour_17\07-17-2019 17_02_08,yes,yes,yes,1,1,1,46,18,15,...,100,1,1,1,1,1,1,1,1,P_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Z_012\Day_1\Hour_9\08-09-2021 09_21_24,yes,yes,yes,1,1,1,46,18,18,...,100,1,1,1,1,1,1,0,-1,Z_012
376,Z_013\Day_4\Hour_21\10-21-2021 21_31_33,yes,yes,yes,1,1,1,1,12,1,...,72,0,1,1,1,1,1,1,1,Z_013
377,Z_013\Day_3\Hour_21\10-20-2021 21_44_00,yes,yes,yes,1,1,1,4,5,2,...,78,0,1,1,1,1,1,1,1,Z_013
378,Z_013\Day_2\Hour_21\10-19-2021 21_12_32,yes,yes,yes,1,1,1,28,19,18,...,83,1,1,1,1,1,1,1,1,Z_013


In [7]:
# Create complete data paths
p_path = "Z:\DyMand Data\Field data\Smartwatch\Partner_in\\"
z_path = "Z:\DyMand Data\Field data\Smartwatch\Zielperson\\"

data['root_dir'] = np.where(data['audio'].str.startswith("P"), p_path, z_path)
data['abs_path'] = data['root_dir'] + data['audio']

In [8]:
data['abs_path'].head(2)

0    Z:\DyMand Data\Field data\Smartwatch\Partner_i...
1    Z:\DyMand Data\Field data\Smartwatch\Partner_i...
Name: abs_path, dtype: object

In [9]:
# Paths

# subdir where data exists
data_paths = data['abs_path'].tolist()


## Sensor data

In [10]:
# Functions to extract sensor data with assistance and adaptation from George's DyMand code

def check_contact(df, column='Contact'):
    counts = df['Contact'].value_counts(normalize=True)
    counts = counts*100
    counts.to_frame()
    sum = 0
    if -1 in list(counts.index):
        sum += counts[-1]
    if 0 in list(counts.index):
        sum += counts[0]
    return sum


def remove_dups(data_frame):
    """Remove duplicate features for a given hour.

    Args:
        data_frame(df): Data frame with features Name, Minutes, Count, Mean, Std, Min, Q1, Median, Q3, Max.

    Returns:
        Data frame with features: Name, Mean, Std, Min, Q1, Median, Q3, Max.
    """
    features = data_frame.sort_values(['Name', 'Minutes'], ascending=[True, True])
    features = features.groupby('Name').nth(-1)
    features.reset_index(inplace=True)
    features = features.drop(['Minutes', 'Count'], axis=1)
    return features


def resample(data_frame, frequency):
    """Resample data.

    Args:
        data_frame(data frame): Data frame with data to standardize.
        frequency(str): Frequency with which we want to resample - e.g. "1s" (for heart rate), "20ms" (for accelerometer, gyroscope).

    Returns:
        Data frame with resampled data.
    """
    data_frame = data_frame.drop(['Date'], axis=1)
    data_frame['Timestamp'] = pd.to_timedelta(data_frame['Timestamp'], unit='ns') #convert time to timedelta
    data_frame.Timestamp = data_frame['Timestamp'].dt.round(frequency) #round timdelta to given frequency 
    data_frame.set_index("Timestamp", inplace=True) # set the time as index
    data_frame = data_frame.resample(frequency).first() #resample to desired frequency
    # data_frame = data_frame.fillna(data_frame.median()) # fill NaN values with median 
    data_frame = data_frame.interpolate() # fill NaN using linear interpolation
    data_frame.reset_index(inplace=True)
    return data_frame 


def remove_outliers(data_frame, column, heartrate = False):

    if np.std(data_frame[column]) != 0:
        data_frame = data_frame[(np.abs(stats.zscore(data_frame[column])) < 1)] ##remove data that is more than 1 std away from mean
    
    if heartrate:
        data_frame = data_frame[data_frame.iloc[:, -1].between(30, 300, inclusive=False)]

    return data_frame



def heart_rate_feat(data_paths, resampling=False, removing_outliers=False, standardizing=False):
    feat = [] # empty list of features with names
    i = 0
    #Loop through folders and subfolders starting at path
    for path in data_paths:
        for file in os.listdir(path):
            #print(file)
            #Check if heart_rate file, read data and find statistics - create a list with name
            if file == "android.sensor.heart_rate.csv":
                filepath = path + os.sep + file # get file
                if os.stat(filepath).st_size != 0:
                    i += 1
                    print()
                    print ("File Count: " + str(i))
                    split_list = filepath.split("\\") 
                    day = split_list[-4]
                    date = split_list[-2]
                    hour = "Hour_" + date[-8:-6]
                    subject = split_list[-5]
                    minute = split_list[-2][-5:-3]
                    if subject != "Demo":
                        combined_name = subject + "_" + day + "_" + hour
                        print(combined_name)
                        print(filepath)
                       
                        heart_rate = pd.read_csv(filepath, names=['Date', 'Timestamp', 'Contact',
                                                                  'HR']) 
                        
                        if check_contact(heart_rate) < 50: # if more than 50% with no contact discard the data
                            
                            features = [combined_name, minute, int(day[-1]), int(date[-8:-7])]
                            if removing_outliers:
                                heart_rate = remove_outliers(heart_rate, 'HR', True)
                            # if standardizing:
                            #    heart_rate = standardize(heart_rate, 'HR')
                            if resampling:
                                heart_rate = resample(heart_rate, '1s')
                            hr_list = heart_rate[
                                'HR'].describe().to_list() 
                            data_range = hr_list[7] - hr_list[3]
                            skewness = stats.skew(heart_rate['HR'])
                            kurtosis = stats.kurtosis(heart_rate['HR'])
                            features.extend(hr_list)
                            features.extend([skewness, kurtosis, data_range])
                            feat.append(features)
       
    features = pd.DataFrame(feat, columns=['Name', 'Minutes', 'HR_Day', 'HR_Hour', 'Count',
                                           'HR_Mean', 'HR_Std', 'HR_Min', 'HR_Q1', 'HR_Q2', 'HR_Q3',
                                           'HR_Max', 'HR_Skew', 'HR_Kurtosis', 'HR_Range'])
    columns = ['HR_Mean', 'HR_Std', 'HR_Min', 'HR_Q1', 'HR_Q2', 'HR_Q3', 'HR_Max', 'HR_Skew', 'HR_Kurtosis',
                           'HR_Range']
    return remove_dups(features)



def accelerometer_feat(data_paths, removing_outliers=False, resampling=False):

    feat = [] # empty list of features with names
    print("Accerometer")
    i = 0
    #Loop through folders and subfolders starting at path
    for path in data_paths:
        for file in os.listdir(path):
            # print(file)
            #Check if accelerometer file, read data and find statistics - create a list with name
            if file == "android.sensor.accelerometer.csv":
                filepath = path + os.sep + file # get file
                if os.stat(filepath).st_size != 0:
                    i += 1
                    print()
                    print ("File Count: " + str(i))
                    split_list = filepath.split("\\") ### check file
                    day = split_list[-4]
                    date = split_list[-2]
                    hour = "Hour_" + date[-8:-6]
                    subject = split_list[-5]
                    minute = split_list[-2][-5:-3]
                    if subject != "Demo":
                        combined_name =  subject + "_" + day + "_" + hour
                        print (combined_name)
                        data = pd.read_csv(filepath, names = ['Date', 'Timestamp', 'Contact', 'x', 'y','z'])
                        
                        # print ("Data length: " + str(len(data)))
                        if len(data) > 0:
                            print("Contact loss: ", check_contact(data))
                            if check_contact(data) < 50:  # if more than 50% with no contact discard the data                                
                                features = [combined_name, minute, int(day[-1]), int(date[-8:-6])]
                                data['m'] = data.apply(lambda row: np.sqrt(row['x']**2 + row['y']**2 + row['z']**2), axis=1)
                                if removing_outliers:
                                    data = remove_outliers(data, 'm')
                                if resampling:
                                    data = resample(data, '20ms')
                                data_list = data['m'].describe().to_list()
                                data_range = data_list[7] - data_list[3]
                                skewness = stats.skew(data['m'])
                                kurtosis = stats.kurtosis(data['m'])
                                features.extend(data_list)
                                features.extend([skewness, kurtosis, data_range])
                                feat.append(features)
    features = pd.DataFrame(feat, columns=['Name', 'Minutes', 'Accelerometer_Day', 'Accelerometer_Hour', 'Count', 'Accelerometer_Mean', 'Accelerometer_Std', 'Accelerometer_Min', 'Accelerometer_Q1', 'Accelerometer_Q2', 'Accelerometer_Q3', 'Accelerometer_Max', 'Accelerometer_Skew', 'Accelerometer_Kurtosis', 'Accelerometer_Range'])
    return remove_dups(features)



In [11]:
hr_pd = heart_rate_feat(data_paths, True, True, True)
hr_pd.to_csv('features/hr_features.csv', sep=',',index=False)


File Count: 1
P_001_Day_2_Hour_16
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_16\07-16-2019 16_44_00\android.sensor.heart_rate.csv

File Count: 2
P_001_Day_2_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_18\07-16-2019 18_24_27\android.sensor.heart_rate.csv


C:\Users\madha\anaconda3\lib\site-packages\ipykernel_launcher.py:77: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.



File Count: 3
P_001_Day_2_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_2\Hour_9\07-16-2019 09_00_26\android.sensor.heart_rate.csv

File Count: 4
P_001_Day_3_Hour_16
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_16\07-17-2019 16_06_50\android.sensor.heart_rate.csv

File Count: 5
P_001_Day_3_Hour_17
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_17\07-17-2019 17_02_08\android.sensor.heart_rate.csv

File Count: 6
P_001_Day_3_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_19\07-17-2019 19_09_54\android.sensor.heart_rate.csv

File Count: 7
P_001_Day_3_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_001\Day_3\Hour_9\07-17-2019 09_01_14\android.sensor.heart_rate.csv

File Count: 8
P_003_Day_2_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_003\Day_2\Hour_19\11-19-2019 19_01_25\android.sensor.heart_rate.csv

File Count: 9
P_003_Day_2_Hour_20
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P


File Count: 59
P_007_Day_0_Hour_15
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_15\02-16-2020 15_03_14\android.sensor.heart_rate.csv

File Count: 60
P_007_Day_0_Hour_16
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_16\02-16-2020 16_02_42\android.sensor.heart_rate.csv

File Count: 61
P_007_Day_0_Hour_17
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_17\02-16-2020 17_35_39\android.sensor.heart_rate.csv

File Count: 62
P_007_Day_0_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_18\02-16-2020 18_11_33\android.sensor.heart_rate.csv

File Count: 63
P_007_Day_0_Hour_22
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_22\02-16-2020 22_15_40\android.sensor.heart_rate.csv

File Count: 64
P_007_Day_0_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_007\Day_0\Hour_9\02-16-2020 09_44_00\android.sensor.heart_rate.csv

File Count: 65
P_007_Day_1_Hour_22
Z:\DyMand Data\Field data\Smartwatch\Part


File Count: 114
P_009_Day_0_Hour_10
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_0\Hour_10\10-18-2020 10_40_49\android.sensor.heart_rate.csv

File Count: 115
P_009_Day_1_Hour_10
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_1\Hour_10\10-12-2020 10_44_00\android.sensor.heart_rate.csv

File Count: 116
P_009_Day_1_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_1\Hour_19\10-12-2020 19_01_29\android.sensor.heart_rate.csv

File Count: 117
P_009_Day_2_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_2\Hour_9\10-13-2020 09_44_00\android.sensor.heart_rate.csv

File Count: 118
P_009_Day_3_Hour_10
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_3\Hour_10\10-14-2020 10_03_32\android.sensor.heart_rate.csv

File Count: 119
P_009_Day_3_Hour_20
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_009\Day_3\Hour_20\10-14-2020 20_44_00\android.sensor.heart_rate.csv

File Count: 120
P_009_Day_4_Hour_10
Z:\DyMand Data\Field data\Smartwat


File Count: 169
P_012_Day_5_Hour_08
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_5\Hour_8\08-13-2021 08_44_12\android.sensor.heart_rate.csv

File Count: 170
P_012_Day_2_Hour_10
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_2\Hour_10\08-10-2021 10_01_00\android.sensor.heart_rate.csv

File Count: 171
P_012_Day_2_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_2\Hour_18\08-10-2021 18_25_48\android.sensor.heart_rate.csv

File Count: 172
P_012_Day_2_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_2\Hour_19\08-10-2021 19_33_58\android.sensor.heart_rate.csv

File Count: 173
P_012_Day_2_Hour_20
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_2\Hour_20\08-10-2021 20_44_05\android.sensor.heart_rate.csv

File Count: 174
P_012_Day_2_Hour_08
Z:\DyMand Data\Field data\Smartwatch\Partner_in\P_012\Day_2\Hour_8\08-10-2021 08_44_04\android.sensor.heart_rate.csv

File Count: 175
P_012_Day_2_Hour_09
Z:\DyMand Data\Field data\Smartwatc


File Count: 224
Z_005_Day_0_Hour_12
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_0\Hour_12\12-01-2019 12_05_54\android.sensor.heart_rate.csv

File Count: 225
Z_005_Day_0_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_0\Hour_19\12-01-2019 19_44_03\android.sensor.heart_rate.csv

File Count: 226
Z_005_Day_2_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_2\Hour_19\11-26-2019 19_09_23\android.sensor.heart_rate.csv

File Count: 227
Z_005_Day_2_Hour_22
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_2\Hour_22\11-26-2019 22_44_03\android.sensor.heart_rate.csv

File Count: 228
Z_005_Day_3_Hour_21
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_3\Hour_21\11-27-2019 21_44_04\android.sensor.heart_rate.csv

File Count: 229
Z_005_Day_3_Hour_22
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_005\Day_3\Hour_22\11-27-2019 22_44_04\android.sensor.heart_rate.csv

File Count: 230
Z_005_Day_3_Hour_06
Z:\DyMand Data\Field data\Smartwa


File Count: 279
Z_007_Day_6_Hour_15
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_15\02-15-2020 15_44_33\android.sensor.heart_rate.csv

File Count: 280
Z_007_Day_6_Hour_16
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_16\02-15-2020 16_44_34\android.sensor.heart_rate.csv

File Count: 281
Z_007_Day_6_Hour_17
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_17\02-15-2020 17_44_34\android.sensor.heart_rate.csv

File Count: 282
Z_007_Day_6_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_18\02-15-2020 18_44_35\android.sensor.heart_rate.csv

File Count: 283
Z_007_Day_6_Hour_19
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_19\02-15-2020 19_44_36\android.sensor.heart_rate.csv

File Count: 284
Z_007_Day_6_Hour_21
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_007\Day_6\Hour_21\02-15-2020 21_44_37\android.sensor.heart_rate.csv

File Count: 285
Z_007_Day_6_Hour_22
Z:\DyMand Data\Field data\Smartwa


File Count: 334
Z_010_Day_2_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_2\Hour_9\02-11-2020 09_33_22\android.sensor.heart_rate.csv

File Count: 335
Z_010_Day_3_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_3\Hour_18\02-12-2020 18_01_11\android.sensor.heart_rate.csv

File Count: 336
Z_010_Day_3_Hour_21
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_3\Hour_21\02-12-2020 21_44_00\android.sensor.heart_rate.csv

File Count: 337
Z_010_Day_3_Hour_08
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_3\Hour_8\02-12-2020 08_03_43\android.sensor.heart_rate.csv

File Count: 338
Z_010_Day_3_Hour_09
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_3\Hour_9\02-12-2020 09_19_04\android.sensor.heart_rate.csv

File Count: 339
Z_010_Day_4_Hour_18
Z:\DyMand Data\Field data\Smartwatch\Zielperson\Z_010\Day_4\Hour_18\02-13-2020 18_34_39\android.sensor.heart_rate.csv

File Count: 340
Z_010_Day_4_Hour_20
Z:\DyMand Data\Field data\Smartwatch

In [18]:
acc_pd = accelerometer_feat(data_paths, True, True)
acc_pd.to_csv('features/acc_features.csv', sep=',',index=False)

Accerometer

File Count: 1
P_001_Day_2_Hour_16
Contact loss:  0

File Count: 2
P_001_Day_2_Hour_18
Contact loss:  0

File Count: 3
P_001_Day_2_Hour_09
Contact loss:  0

File Count: 4
P_001_Day_3_Hour_16
Contact loss:  0

File Count: 5
P_001_Day_3_Hour_17
Contact loss:  0

File Count: 6
P_001_Day_3_Hour_19
Contact loss:  0

File Count: 7
P_001_Day_3_Hour_09
Contact loss:  0

File Count: 8
P_003_Day_2_Hour_19
Contact loss:  0

File Count: 9
P_003_Day_2_Hour_20
Contact loss:  0

File Count: 10
P_003_Day_3_Hour_19
Contact loss:  0

File Count: 11
P_003_Day_4_Hour_19
Contact loss:  0

File Count: 12
P_003_Day_5_Hour_19
Contact loss:  0

File Count: 13
P_003_Day_6_Hour_10
Contact loss:  0

File Count: 14
P_003_Day_6_Hour_12
Contact loss:  0

File Count: 15
P_003_Day_6_Hour_13
Contact loss:  0

File Count: 16
P_003_Day_6_Hour_09
Contact loss:  0

File Count: 17
P_004_Day_0_Hour_10
Contact loss:  0

File Count: 18
P_004_Day_0_Hour_15
Contact loss:  0

File Count: 19
P_004_Day_0_Hour_18
Contact

Contact loss:  0

File Count: 155
P_011_Day_4_Hour_09
Contact loss:  0

File Count: 156
P_012_Day_4_Hour_18
Contact loss:  0

File Count: 157
P_012_Day_4_Hour_20
Contact loss:  0

File Count: 158
P_012_Day_4_Hour_09
Contact loss:  0

File Count: 159
P_012_Day_3_Hour_18
Contact loss:  0

File Count: 160
P_012_Day_3_Hour_19
Contact loss:  0

File Count: 161
P_012_Day_3_Hour_08
Contact loss:  0

File Count: 162
P_012_Day_6_Hour_15
Contact loss:  0

File Count: 163
P_012_Day_6_Hour_18
Contact loss:  0

File Count: 164
P_012_Day_6_Hour_19
Contact loss:  0

File Count: 165
P_012_Day_6_Hour_20
Contact loss:  0

File Count: 166
P_012_Day_5_Hour_10
Contact loss:  0

File Count: 167
P_012_Day_5_Hour_18
Contact loss:  0

File Count: 168
P_012_Day_5_Hour_20
Contact loss:  0

File Count: 169
P_012_Day_5_Hour_08
Contact loss:  0

File Count: 170
P_012_Day_2_Hour_10
Contact loss:  0

File Count: 171
P_012_Day_2_Hour_18
Contact loss:  0

File Count: 172
P_012_Day_2_Hour_19
Contact loss:  0

File Count

Contact loss:  0

File Count: 307
Z_009_Day_3_Hour_10
Contact loss:  0

File Count: 308
Z_009_Day_3_Hour_20
Contact loss:  0

File Count: 309
Z_009_Day_4_Hour_10
Contact loss:  0

File Count: 310
Z_009_Day_5_Hour_10
Contact loss:  0

File Count: 311
Z_009_Day_5_Hour_19
Contact loss:  0

File Count: 312
Z_009_Day_5_Hour_20
Contact loss:  0

File Count: 313
Z_009_Day_6_Hour_11
Contact loss:  0

File Count: 314
Z_009_Day_6_Hour_12
Contact loss:  0

File Count: 315
Z_009_Day_6_Hour_16
Contact loss:  0

File Count: 316
Z_009_Day_6_Hour_17
Contact loss:  0

File Count: 317
Z_009_Day_6_Hour_18
Contact loss:  0

File Count: 318
Z_009_Day_6_Hour_19
Contact loss:  0

File Count: 319
Z_009_Day_6_Hour_20
Contact loss:  0

File Count: 320
Z_010_Day_0_Hour_16
Contact loss:  0

File Count: 321
Z_010_Day_0_Hour_17
Contact loss:  0

File Count: 322
Z_010_Day_0_Hour_18
Contact loss:  0

File Count: 323
Z_010_Day_0_Hour_19
Contact loss:  0

File Count: 324
Z_010_Day_0_Hour_20
Contact loss:  0

File Count

In [19]:
hr_pd.shape

(380, 13)